In [ ]:
import imports
from traces_numpy import *
%matplotlib inline

In [ ]:
iZIP7   = 'data/SNOLAB_TES/iZIP7-TES_51231118_000000.root'
iZIP7Si = 'data/SNOLAB_TES/iZIP7Si-TES_51231118_000000.root'
HV100mm = 'data/SNOLAB_TES/HV100mm-TES_51231118_000000.root'
HVSi    = 'data/SNOLAB_TES/HV100mmSi-TES_51231118_000000.root'

In [ ]:
# All the SNOLAB channels should have the same baseline and same scaling
def TESscaling(file):
    chans = getChannels(file, "TES")
    
    # Get channel energy from "g4dmcEvent" branch
    data = root_numpy.root2array(file, treename="G4SimDir/g4dmcEvent", branches=['PhononE'])
    phononE = data['PhononE'][0]         # Pull first event array out of data structure
    
    # Trace binning, useful for finding average baseline
    binning = getBinning(file, "TES")
    start = int(-binning['T0']/binning['BinWidth'])   # Index of start of signal

    traceRaw = getRawTES(file)['PAS1'][0]
    traceTES = getTES(file)['PAS1'][0]

    I0 = np.mean(traceRaw[10:start])
    Ipeak = np.max(traceTES)
    IvsE = Ipeak / phononE[chans.index('PAS1')]  # PhononE is just a simple array
    
    # FIXME:  Get trace units from TTree, instead of assuming them
    print("Ipeak = {} uA @ {} eV".format(Ipeak, phononE[chans.index('PAS1')]))
    print(f"I0 = {I0} uA")
    print(f"I/E = {IvsE} uA/eV")

In [ ]:
# All the SNOLAB channels should have the same baseline and same scaling
def TESshape(file):
    chans = getChannels(file, "TES")
    
    # Trace binning parameters to find index of start of signal
    binning = getBinning(file, "TES")
    start = int(-binning['T0']/binning['BinWidth'])

    traceTES = getTES(file)['PAS1'][0][start:]
    bins = getBins(file, "TES")['PAS1'][start:]/1e6   # Convert to seconds
    
    def tesShape(t, A, rise, fall):
        return A*(np.exp(-t/fall)-np.exp(-t/rise))

    tesfit = curve_fit(tesShape, bins, traceTES, p0=[32,100e-6,1000e-6])
    print("Fit output and covariance matrix")
    print(tesfit[0])
    print(tesfit[1])
    [A, rise, fall] = tesfit[0]
    print("Rise time: {} us".format(rise*1e6))
    print("Fall time: {} us".format(fall*1e6))

In [ ]:
TESscaling(iZIP7)
# Shape fitting isn't working yet
TESshape(iZIP7)

In [ ]:
TESscaling(iZIP7Si)

In [ ]:
TESscaling(HV100mm)

In [ ]:
TESscaling(HVSi)